In [32]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import string
import os
import time

driver = webdriver.Firefox()
driver.get('https://csf.tools/controlset/nist800-53r5/?name_filter=&baseline%5B%5D=nist800-53r5-high')



In [33]:
table_data = []
last_set = None
has_more_data = True

try:
    while [has_more_data]:
        table = WebDriverWait(driver=driver, timeout=10).until(
            EC.presence_of_element_located(locator=(By.ID, 'controlset-nist800-53r5'))
        )
        for row in table.find_elements(By. CSS_SELECTOR, 'tr'):
            columns = row.find_elements(By.CSS_SELECTOR, 'td')
            if len(columns) < 1:
                # likly the header, we should do more here later
                pass
            else:
                # Main code flow
                if "controlset-element-enhancement" in row.get_attribute("class"):
                    id = f'{last_set}{columns[0].text}'
                elif "controlset-row" in row.get_attribute("class"):
                    last_set = columns[0].text
                    id = last_set
                stride_column = columns[6].find_elements(By.CSS_SELECTOR, "span")
                stride_mapping = {}
                for letter in stride_column:
                    stride_mapping[letter.text] = "controlset-threat-yes" in letter.get_attribute("class")
                row_data = {
                    "id": id,
                    "name":  columns [1]. text,
                    "low":  True if  "•" in columns [2]. text else False,
                    "moderate": True if "•" in columns[3]. text else False,
                    "high": True if "•" in columns[4].text else False,
                    "spoofing": stride_mapping['S'],
                    "tampering":  stride_mapping['T'],
                    "repudiation": stride_mapping['R'],
                    "information_disclosure": stride_mapping['I'],
                    "denial_of_service":  stride_mapping['D'],
                    "elevation_of_privilege": stride_mapping['E'],
                    "lateral_movement":  stride_mapping['LM']
                }
                table_data.append(row_data)
        next_page = driver.find_elements(By.CSS_SELECTOR, "a.next.page-numbers")
        if len(next_page) > 0:
            print("Going to the next page")
            next_page.pop().click()
        else:
            print("out of data")
            has_more_data = False
            break

finally:
    driver.close()

Going to the next page
Going to the next page
Going to the next page
Going to the next page
Going to the next page
Going to the next page
Going to the next page
out of data


In [34]:
import csv
with open(file="80053-STRIDELM-Mapping.csv", mode="w", newline="", encoding="utf-8") as output:
    writer = csv.DictWriter(f=output, fieldnames=table_data[0].keys())
    writer.writeheader()
    writer.writerows(table_data)